In [13]:
import random as rnd
import queue
import statistics as stat
from tabulate import tabulate
import matplotlib.pyplot as plt
import numpy as np
from math import sqrt
import random
import copy

# This program will simulate customers going through a Theme Park, with attractions for each land to be visited in a random manner.
# Future implementation is to implement ML techniques in choosing lands and attractions.

### PARAMETERS ###
param = {'_lambda': 0.5, #Inter Arrival Time (IAT) in seconds (expovar)
         'Tot_Sim_Time': 1*60*60, #Total time of simulation (in seconds)
         'TTC_TH': 60*30, #Time Til Close Threshold (in seconds)
         '_STsec': 1.2 #Security Check service time in seconds (expovar)
        }

### STATE VARS ###
state_var = {'cust_in_TP': 0,
             'SBsec': False,
             'qsec': 0,
             'ID_list_LC':[]
            } #Number of customers currently in the theme park

### OUTVARS ###
out_var = {'Count_Arrival_Ev': 0,
           'TPA': [],
           'TPD': [],
           'sec_arr':[],
           'sec_dep': []
          }

### ATTRIBUTES ###
cust_att= [] #Nested list denoting if customer has visited rides - ELABORATE?
attr = [] #All customer attributes shall be contained in this list, where their index is the ID

### SYSTEMIZING THE DEFINITION OF PARAMS, STATE_VARS, OUT_VARS AND CUST_ATT ###

LN = 3 #Number of lands in the Theme Park
AN = 3 #Number of attractions in each land

for i in range(1, LN+1):
    cust_att.append([False]*AN) #Create an attribute list template for customers arriving
    myKeyLandsArr = 'arrs_L' + str(i) #Record Arrival times in each land
    out_var[myKeyLandsArr] = []
    myKeyLandsDep = 'deps_L' + str(i) #Record Departure times in each land
    out_var[myKeyLandsDep] = []
    myKeyIDLs = 'ID_list_L' + str(i) #Record ID numbers of people in Land #
    state_var[myKeyIDLs] = []
    myKeyIDACs = 'ID_list_AC' + str(i) #Record ID numbers of customers to be transferred to attraction choice func.
    state_var[myKeyIDACs] = []
    for j in range(1, AN+1):
        myKey1 = 'ST' + str(i) + str(j) #Service time for each attraction, arbitrary random times for now.
        param[myKey1] = 250 + 50*j
        myKey2 = 'AC' + str(i) + str(j) #Attraction customer capacity,  arbitrary random numbers for now.
        param[myKey2] = 18 + 3*j
        myKey3 = 'SB' + str(i) + str(j) #Server Busy state
        state_var[myKey3] = False
        myKey4 = 'q' + str(i) + str(j) #Queue length
        state_var[myKey4] = 0
        myKey7 = 'ID_list_A' + str(i) + str(j) #Queue length
        state_var[myKey7] = []
        myKey5 = 'arrs_A' + str(i) + str(j) #Arrival times for each attraction
        out_var[myKey5] = []
        myKey6 = 'deps_A' + str(i) + str(j) #Departure times for each attraction
        out_var[myKey6] = []      

### SAMPLE PATH (SP) ###
global SP
global out_var
SP = []

In [14]:
# Define the global event list
evList = queue.PriorityQueue() #Try PriorityQueue? (no prioriy)

### Define the Event Generators ###
def get_next_arrival_event_TP (clock, param):
    out_var['Count_Arrival_Ev'] += 1
    iat = rnd.expovariate(param['_lambda'])
    ev = (clock+iat, "Arrival to Theme Park", arrival_event_handler_TP)
    return ev    
    
def get_start_security_check_event (clock, param):
    ev = (clock, "Security Check 1 Start", security_check_event_handler)
    return ev

def get_land_choice (clock, param, cust_id): #For any customer id as an argument, add this customer Id to its corresponding event hanlder list as follows:
    state_var['ID_list_LC'].append(cust_id)
    ev = (clock, "Land Choice for Customer#(" + str(cust_id)+")", land_choice_event_handler)
    return ev

def get_enter_land_1_event(clock, param, cust_id):
    state_var['ID_list_L1'].append(cust_id)
    ev = (clock, "Customer Enters Land 1, #(" + str(cust_id)+")", land_1_arr_event_handler)
    return ev

def get_att_1x_choice (clock, param, cust_id): #AC1 - Attraction Choice for land 1
    state_var['ID_list_AC1'].append(cust_id)
    ev = (clock, "Attraction 1x Choice for Customer#(" + str(cust_id)+")", att_1x_choice_event_handler)
    return ev

def get_start_att_11_event(clock, param, cust_id):
    ev = (clock, "Customer Enters Attraction 11, #(" + str(cust_id)+")", start_att_11_event_handler)
    return(ev)

def get_start_att_12_event(clock, param, cust_id):
    ev = (clock, "Customer Enters Attraction 12, #(" + str(cust_id)+")", start_att_12_event_handler)
    return(ev)

def get_start_att_13_event(clock, param, cust_id):
    ev = (clock, "Customer Enters Attraction 13, #(" + str(cust_id)+")", start_att_13_event_handler)
    return(ev)

def get_enter_land_2_event(clock, param, cust_id):
    state_var['ID_list_L2'].append(cust_id)
    ev = (clock, "Customer Enters Land 2, #(" + str(cust_id)+")", land_2_arr_event_handler)
    return ev

def get_att_2x_choice (clock, param, cust_id): #AC2 - Attraction Choice for land 2
    state_var['ID_list_AC2'].append(cust_id)
    ev = (clock, "Attraction 2x Choice for Customer#(" + str(cust_id)+")", att_2x_choice_event_handler)
    return ev

def get_start_att_21_event(clock, param, cust_id):
    ev = (clock, "Customer Enters Attraction 21, #(" + str(cust_id)+")", start_att_21_event_handler)
    return(ev)

def get_start_att_22_event(clock, param, cust_id):
    ev = (clock, "Customer Enters Attraction 22, #(" + str(cust_id)+")", start_att_22_event_handler)
    return(ev)

def get_start_att_23_event(clock, param, cust_id):
    ev = (clock, "Customer Enters Attraction 23, #(" + str(cust_id)+")", start_att_23_event_handler)
    return(ev)

def get_enter_land_3_event(clock, param, cust_id):
    state_var['ID_list_L3'].append(cust_id)
    ev = (clock, "Customer Enters Land 3, #(" + str(cust_id)+")", land_3_arr_event_handler)
    return ev

def get_att_3x_choice (clock, param, cust_id): #AC3 - Attraction Choice for land 3
    state_var['ID_list_AC3'].append(cust_id)
    ev = (clock, "Attraction 3x Choice for Customer#(" + str(cust_id)+")", att_3x_choice_event_handler)
    return ev

def get_start_att_31_event(clock, param, cust_id):
    ev = (clock, "Customer Enters Attraction 31, #(" + str(cust_id)+")", start_att_31_event_handler)
    return(ev)

def get_start_att_32_event(clock, param, cust_id):
    ev = (clock, "Customer Enters Attraction 32, #(" + str(cust_id)+")", start_att_32_event_handler)
    return(ev)

def get_start_att_33_event(clock, param, cust_id):
    ev = (clock, "Customer Enters Attraction 33, #(" + str(cust_id)+")", start_att_33_event_handler)
    return(ev)


########### GET BACK TO THE DEPS ############

def get_next_departure_event_security_check (clock, param):
    st = rnd.expovariate(param['_STsec'])
    ev = (clock+st, "Security Check 1 End", departure_event_handler_security_check)
    return ev

#### MODIFY! ###
def get_exit_land_1_event(clock, param, cust_id):
    ###corresponding id list
    ev = (clock, "Customer Exits Land 1, #(" + str(cust_id)+")", land_1_arr_event_handler)
    return ev

def get_depart_att_11_event(clock, param):
    st = param['ST11']
    ev = (clock+st, "Customers Leaving Attraction 11", depart_att_11_event_handler)
    return(ev)

def get_depart_att_12_event(clock, param):
    st = param['ST12']
    ev = (clock+st, "Customers Leaving Attraction 12", depart_att_12_event_handler)
    return(ev)

def get_depart_att_13_event(clock, param):
    st = param['ST13']
    ev = (clock+st, "Customers Leaving Attraction 13", depart_att_13_event_handler)
    return(ev)

def get_next_departure_event_TP (clock, param, cust_id):
    out_var['TPD'].append([clock, cust_id])
    ev = (clock, "Departure from Theme Park, #(" + str(cust_id)+")", departure_event_handler_TP)
    return ev


In [15]:
# Define Event handlers

# ARRIVALS ___________________
def arrival_event_handler_TP (clock, state_var, out_var, param, att): # Commented lines are for future use
    if (True):   # Check if time til close is greater than threshold
        state_var['cust_in_TP'] += 1                # Increase number of customers in theme park
        out_var['TPA'].append(clock)                # Record arrival time to Theme park        
        state_var['qsec'] += 1                # Essentially, make customer enter queue 1
        if (state_var['SBsec'] == False) and (state_var['qsec'] >= 1): #If conditions are satisfied, start ride
            schedule_event(get_start_security_check_event(clock, param))        
    if (param['Tot_Sim_Time']-clock) > param['TTC_TH']: #Check if time til close is greater than threshold
        schedule_event(get_next_arrival_event_TP(clock, param)) #Generate New Arrival to keep program running

def security_check_event_handler (clock, state_var, out_var, param, att):
    state_var['Bsec'] = True #Make the security guard busy
    state_var['qsec'] -= 1    #Remove person doing the security check from queue
    out_var['sec_arr'].append(clock)
    schedule_event(get_next_departure_event_security_check(clock, param))
    
def land_choice_event_handler (clock, state_var, out_var, param, att):
    cust_id = state_var['ID_list_LC'].pop()
    land_options = [] #Initialize land options, ie. lands customer can visit
    for land_index in range(0, LN):
        first_element = attr[cust_id][land_index][0] #Grab first attraction state
        for state in attr[cust_id][land_index]:
            if (first_element != state):
                raise Exception("Customer has not visited all rides") #Remove in next revision
        land_options.append(not(first_element)) #append the negation of of ride, as in, if customer visited attractions in land 1, it is not availabe as an option
    choice_list = [] #Initialize choice list, this list contains only the land numbers that are permitted to be entered
    for i in range(len(land_options)):
        if land_options[i]: #If land is availabe, ie. true
            choice_list.append(i+1) #append land number in choice list from land options list
    if len(choice_list) == 0:
        schedule_event(get_next_departure_event_TP(clock, param, cust_id))
    else:
        final_choice = random.choice(choice_list)
        if final_choice == 1:
            schedule_event(get_enter_land_1_event(clock, param, cust_id))
        if final_choice == 2:
            schedule_event(get_next_departure_event_TP(clock, param, cust_id)) #Remove in next rev
            #schedule_event(get_enter_land_2_event(clock, param, cust_id))
        if final_choice == 3:
            schedule_event(get_next_departure_event_TP(clock, param, cust_id)) #Remove in next rev
            #schedule_event(get_enter_land_3_event(clock, param, cust_id))

def land_1_arr_event_handler (clock, state_var, out_var, param, att):
    cust_id = state_var['ID_list_L1'][-1]      #COULD RAISE ERRORS, NEEDS BETTER PROGRAMMING!
    out_var['arrs_L1'].append([clock, cust_id])#DEPENDANT ON ABOVE!
    schedule_event(get_att_1x_choice(clock, param, cust_id))
        
def att_1x_choice_event_handler (clock, state_var, out_var, param, att):
    cust_id = state_var['ID_list_AC1'].pop()
    att_options = [] #Initialize attraction options, ie. attractions customer can visit
    choice_list = [] #Initialize choice list, this list contains only the attraction numbers that are permitted to be entered
    for att_index in range(0, AN):
        att_options.append(not(attr[cust_id][0][att_index])) #append the negation of attraction, as in, if customer visited attraction already, it is not availabe as an option
    for i in range(len(att_options)):
        if att_options[i]: #If land is availabe, ie. true
            choice_list.append(i+1) #append land number in choice list from land options list
    if len(choice_list) == 0:
        schedule_event(get_next_departure_event_TP(clock, param, cust_id)) # SHALL BE EXIT LAND 1 !!!!
    else:
        final_choice = random.choice(choice_list)
        if final_choice == 1:
            out_var['arrs_A11'].append([clock, cust_id])
            state_var['q11'] +=1
            state_var['ID_list_A11'].append(cust_id)
            schedule_event(get_start_att_11_event(clock, param, cust_id))
        if final_choice == 2:
            out_var['arrs_A12'].append([clock, cust_id])
            state_var['q12'] +=1
            state_var['ID_list_A12'].append(cust_id)
            schedule_event(get_start_att_12_event(clock, param, cust_id))
        if final_choice == 3:
            out_var['arrs_A13'].append([clock, cust_id])
            state_var['q13'] +=1
            state_var['ID_list_A13'].append(cust_id)
            schedule_event(get_start_att_13_event(clock, param, cust_id))

def start_att_11_event_handler (clock, state_var, out_var, param, att):
    if (state_var['q11'] >= param['AC11'] and state_var['SB11'] == False):
        state_var['SB11'] = True
        state_var['q11'] -= param['AC11']
        schedule_event(get_depart_att_11_event(clock, param))
        
def start_att_12_event_handler (clock, state_var, out_var, param, att):
    if (state_var['q12'] >= param['AC12'] and state_var['SB12'] == False):
        state_var['SB12'] = True
        state_var['q12'] -= param['AC12']
        schedule_event(get_depart_att_12_event(clock, param))

def start_att_13_event_handler (clock, state_var, out_var, param, att):
    if (state_var['q13'] >= param['AC13'] and state_var['SB13'] == False):
        state_var['SB13'] = True
        state_var['q13'] -= param['AC13']
        schedule_event(get_depart_att_13_event(clock, param))
    

# DEPARTURES ___________________
def departure_event_handler_security_check (clock, state_var, out_var, param, att):
    state_var['SBsec'] = False # Make the ride free
    #CHECK OUTVARS FOR ARRIVALS, AND DEPS
    out_var['sec_dep'].append(clock)
    if state_var['qsec'] >= 1:
        schedule_event(get_start_security_check_event(clock, param))
    ### ASSIGN AN ATTRIBUTE LIST FOR CUSTOMERS LEAVING THE SECURITY GATE, AND ADD CHOICE FUNCTION TO CHOOSE WHICH LAND CUSTOMER IS GOING TO NEXT ###
    ### ID ASSIGNMENT OF CUSTOMERS IS DONE AFTER SECURITY GATE
    attr.append(copy.deepcopy(cust_att))
    schedule_event(get_land_choice(clock, param, len(attr)-1)) # "len(attr)-1" IS THE ID NUMBER OF CUSTOMER
    
def depart_att_11_event_handler (clock, state_var, out_var, param, att):
    state_var['SB11'] = False # Make the ride free
    for i in range (0, param['AC11']):
        cust_id = state_var['ID_list_A11'].pop(0)
        attr[cust_id][0][0]=True
        out_var['deps_A11'].append([clock, cust_id])
        schedule_event(get_att_1x_choice(clock, param, cust_id))
    if (state_var['q11'] >= param['AC11'] and state_var['SB11'] == False):
        schedule_event(get_start_att_11_event(clock, param, cust_id))
        
def depart_att_12_event_handler (clock, state_var, out_var, param, att):
    state_var['SB12'] = False # Make the ride free
    for i in range (0, param['AC12']):
        cust_id = state_var['ID_list_A12'].pop(0)
        attr[cust_id][0][1]=True
        out_var['deps_A12'].append([clock, cust_id])
        schedule_event(get_att_1x_choice(clock, param, cust_id))
    if (state_var['q12'] >= param['AC12'] and state_var['SB12'] == False):
        schedule_event(get_start_att_12_event(clock, param, cust_id))
        
def depart_att_13_event_handler (clock, state_var, out_var, param, att):
    state_var['SB13'] = False # Make the ride free
    for i in range (0, param['AC13']):
        cust_id = state_var['ID_list_A13'].pop(0)
        attr[cust_id][0][2]=True
        out_var['deps_A13'].append([clock, cust_id])
        schedule_event(get_att_1x_choice(clock, param, cust_id))
    if (state_var['q13'] >= param['AC13'] and state_var['SB13'] == False):
        schedule_event(get_start_att_13_event(clock, param, cust_id))
        
def departure_event_handler_TP (clock, state_var, out_var, param, att):
    state_var['cust_in_TP'] -= 1 

In [16]:
# Insert an event into the event list		
def schedule_event(ev):
    global evList
    evList.put(ev)

def CheckConsistency (state_var, param): # Checks if state variables are consistent, used for debugging
    isconsistent = (state_var['q_length_1'] + state_var['q_length_2'] + state_var['q_length_3'] + 
    state_var['server_busy_1']*param['_RC1'] + state_var['server_busy_2']*param['_RC2'] + 
    state_var['server_busy_3']*param['_RC3'])
    if state_var['cust_in_TP'] == isconsistent:
        return True
    else:
        return False


# Main simulation function
def sim(state_var, out_var, param):
    global evList
    global SP
    clock = 0
    evList = queue.PriorityQueue()
    # Reset state and output variables
    
    #ENTER ALL STATE VARS BELOW!!! 
    # DONT FORGET TO RESET STATE VARS
    # Initialize (seed) the RNG	
    #rnd.seed(777)
    
    # Insert initial event
    ev = get_next_arrival_event_TP(clock, param)
    schedule_event(ev)
    
    # Start simulation
    while not evList.empty():
        ev = evList.get()
        clock = ev[0] #Update time
        ev[2](clock, state_var, out_var, param, attr)
        SP.append([len(SP), round(clock,2),ev[1]])
def main():
    
    
    print("Input Parameters:")
    for key in param:
        print(key, '->', param[key])
    rnd.seed(777) # Reseed RNG
    sim(state_var, out_var, param)
    print(tabulate(SP, headers=["ID", "Time","Event"])) 

if __name__ == '__main__':
    main()

Input Parameters:
_lambda -> 0.5
Tot_Sim_Time -> 3600
TTC_TH -> 1800
_STsec -> 1.2
ST11 -> 300
AC11 -> 21
ST12 -> 350
AC12 -> 24
ST13 -> 400
AC13 -> 27
ST21 -> 300
AC21 -> 21
ST22 -> 350
AC22 -> 24
ST23 -> 400
AC23 -> 27
ST31 -> 300
AC31 -> 21
ST32 -> 350
AC32 -> 24
ST33 -> 400
AC33 -> 27
  ID     Time  Event
----  -------  ---------------------------------------
   0     0.52  Arrival to Theme Park
   1     0.52  Security Check 1 Start
   2     0.9   Security Check 1 End
   3     0.9   Land Choice for Customer#(0)
   4     0.9   Departure from Theme Park, #(0)
   5     1.7   Arrival to Theme Park
   6     1.7   Security Check 1 Start
   7     2.34  Security Check 1 End
   8     2.34  Land Choice for Customer#(1)
   9     2.34  Customer Enters Land 1, #(1)
  10     2.34  Attraction 1x Choice for Customer#(1)
  11     2.34  Customer Enters Attraction 12, #(1)
  12     5.26  Arrival to Theme Park
  13     5.26  Security Check 1 Start
  14     6.07  Security Check 1 End
  15     6.07  Lan

In [17]:
attr[1]

[[True, True, True], [False, False, False], [False, False, False]]

In [18]:
for i in range (len(out_var['TPD'])):
    if out_var['TPD'][i][1] == 1:
        print(i)

678


In [19]:
(out_var['TPD'][678][0] - out_var['TPA'][1])/60

48.22730923137323